### Preparing a ligand with `openmmforcefields` and Interchange

[`openmmforcefields`](https://github.com/openmm/openmmforcefields) is a Python package that provides OpenMM implementations of some small molecule force fields via small molecule template generators.

### Validating the implementation of SMIRNOFF force fields

`openmmforcefields` provides SMIRNOFF force fields via its infrastructure, internally calling the OpenFF Toolkit and using SMIRNOFF force fields provided by its `ForceField` class. Before doing novel things, let's validate that this implementation provides the same result as directly using OpenFF tools.

In [ ]:
from openff.toolkit import Molecule
from openff.units import unit
from openff.units.openmm import ensure_quantity
from openmm.app import ForceField as OpenMMForceField
from openmmforcefields.generators import SMIRNOFFTemplateGenerator

from openff.interchange.drivers.openmm import _get_openmm_energies

molecule = Molecule.from_smiles("O=S(=O)(N)c1c(Cl)cc2c(c1)S(=O)(=O)NCN2")
molecule.generate_conformers(n_conformers=1)

smirnoff = SMIRNOFFTemplateGenerator(molecules=molecule)
forcefield = OpenMMForceField()
forcefield.registerTemplateGenerator(smirnoff.generator)

system = forcefield.createSystem(
    molecule.to_topology().to_openmm(),
    nonbondedCutoff=ensure_quantity(0.9 * unit.nanometer, "openmm"),
    switchDistance=ensure_quantity(0.8 * unit.nanometer, "openmm"),
)

`openmmforcefields` has provided us an (OpenMM) force field with SMIRNOFF parameters included as a tempalte generator. The end goal of this setup is to create an `openmm.System`, which we can get a single-point energy of.

In [ ]:
from openff.units import unit
from openff.units.openmm import ensure_quantity

from openff.interchange.drivers.openmm import _get_openmm_energies

energy_openmmforcefields = _get_openmm_energies(
    system,
    box_vectors=None,
    positions=ensure_quantity(molecule.conformers[0], "openmm"),
    platform="Reference",
)

energy_openmmforcefields

We can compare this to an analogous series of operations that use OpenFF tools (the toolkit's `ForceField` class and Interchange) to create an `openmm.System` that one would hope has identical contents.

In [ ]:
from openff.toolkit import ForceField

from openff.interchange import Interchange

sage = ForceField("openff_unconstrained-2.0.0.offxml")
interchange = Interchange.from_smirnoff(sage, [molecule])

energy_openff = _get_openmm_energies(
    interchange.to_openmm(combine_nonbonded_forces=True),
    box_vectors=None,
    positions=ensure_quantity(molecule.conformers[0], "openmm"),
    platform="Reference",
)

energy_openff

Manually inspecting the energies shows zero or marginal differences between them. We can also programmically compare these `EnergyReport` objects with `.compare`, which raises an error if there are significant differences, or `.diff`, which reports differences whether or not they are significant.

In this case, nonbonded energies may differ slightly due to differences in how `openmmforcefields` treats non-bonded settings and/or rounding errors in PME calculations.

In [ ]:
energy_openff.compare(
    energy_openmmforcefields,
    {"Nonbonded": 0.002 * unit.kilojoule_per_mole},
)

energy_openff.diff(energy_openmmforcefields)

### Comparing to GAFF parametrization

`openmmforcefields` also provides a (partial) implementation of GAFF, using AM1-BCC in place of RESP to generate partial charges. The API for using `GAFFTemplateGenerator` is analogous to using `SMIRNOFFTemplateGenerator`.

In [ ]:
from openmmforcefields.generators import GAFFTemplateGenerator

gaff = GAFFTemplateGenerator(molecules=molecule)
forcefield_gaff = OpenMMForceField()
forcefield_gaff.registerTemplateGenerator(gaff.generator)

system_gaff = forcefield_gaff.createSystem(molecule.to_topology().to_openmm())

From here, one could use `from_openmm` to import this `openmm.System` into an `Interchange` object, which then could be combined with other `Interchange` objects generated by different paths.

In [ ]:
from openff.interchange.interop.openmm import from_openmm

from_openmm(
    molecule.to_topology().to_openmm(),
    system_gaff,
    box_vectors=None,
    positions=ensure_quantity(molecule.conformers[0], "openmm"),
)